# 1 Assignment 3

1. Generate a function named **beta_OLS**. This function must return the estimated beta using OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The function's output must be a **pd.DataFrame** (`n`, 1) where `n` is the total number of regressors ( it includes the **Intercept** when `intercept` is `True`). The column should be named as **Coef.** and the row index should be named as the original name of the columns in `X` input. Your results should look like `beta_OLS_output`. Apply your function to find $\widehat{\boldsymbol{\beta}}^{(OLS)}$ of the equation below. Use `data`.  **Hint: Use NumPy to generate the OLS beta and check [this link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) to change the index name of a pd.Dataframe, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\beta}}^{(OLS)} = \left( \mathbf{X}^\top \mathbf{X}\right)^{-1} \mathbf{X}^\top \mathbf{Y}
$$


$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [1]:
import pandas as pd
import numpy as np
import pickle
import scipy.stats as stats

In [2]:
greene = pd.read_csv(r"../../_data/christensen_greene_f4.csv")
ct = greene.COST.values
q = greene.Q.values
p1 = greene.PL.values
p2 = greene.PF.values
p3 = greene.PK.values

In [3]:
file_to_open = open( r'../../_data/data_greene.pkl' , "rb" )
data = pickle.load( file_to_open )

In [4]:
X = data.iloc[ :, 1: ]
X

,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
0,2.079442,4.324077,18.371538,6.010359,8.678634,8.834842,2.890372,4.173541,39.027219,4.177124,8.709221,12.767989,18.436287,6.031542
1,6.767343,45.796933,61.127805,20.624885,28.577410,9.032763,3.047708,4.222840,40.795401,4.644262,8.916190,13.764611,19.071958,6.434992
2,7.252762,52.602563,65.146469,27.026810,26.878966,8.982297,3.726416,3.706032,40.340833,6.943088,6.867335,16.735889,16.644338,6.905108
3,4.174387,17.425509,37.994654,13.989505,15.526556,9.101852,3.351272,3.719481,41.421852,5.615511,6.917271,15.251388,16.927084,6.232496
4,5.686975,32.341689,51.263140,20.863674,24.316554,9.014131,3.668677,4.275832,40.627277,6.729595,9.141372,16.534966,19.271456,7.843324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,6.852877,46.961918,63.543838,27.006249,25.870001,9.272579,3.940863,3.775057,42.990358,7.765201,7.125528,18.270982,17.502257,7.438492
154,5.932245,35.191533,52.877557,20.820004,25.542382,8.913583,3.509633,4.305685,39.725977,6.158762,9.269463,15.641703,19.189541,7.555688
155,5.968708,35.625470,51.749248,22.671977,26.025919,8.670093,3.798473,4.360394,37.585255,7.214200,9.506519,16.466559,18.902512,8.281421
156,8.578665,73.593485,78.650266,32.031471,37.375623,9.168125,3.733853,4.356811,42.027255,6.970828,9.490903,17.116214,19.971895,8.133846


In [5]:
y = data.ln_cost
y

0     -1.546463
1      1.112745
2      2.241337
3     -0.273648
4      0.814789
         ...   
153    1.921222
154    1.324552
155    1.381834
156    3.407444
157    4.217391
Name: ln_cost, Length: 158, dtype: float64

In [6]:
file_to_open = open( r'../../_data/beta_OLS_output.pkl' , "rb" )
beta_OLS_output = pickle.load( file_to_open )
beta_OLS_output

,Coef.
Intercept,-76.259259
lnq,-1.080425
(lnq)^2,0.026489
(lnq)(lnp1),0.131041
(lnq)(lnp2),0.058652
(lnq)(lnp3),0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797
(lnp1)^2,-1.538527


In [7]:
def beta_OLS( X : pd.DataFrame , y : pd.Series , intercept = True ):
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( 'X is not a pd.DataFrame' )
    
    if not isinstance( y, pd.Series ):
        raise TypeError( 'y is not a pd.Series' )
    
    if not isinstance( intercept, bool ):
        raise TypeError( 'intercept is not a bool' )
        
        
    names = list(X.columns)


    if (intercept == True):

        X1 = np.hstack(( np.ones(len(X)).reshape(-1, 1 ) , X)) 
    
        beta_ols = pd.DataFrame( np.linalg.inv(X1.T @ X1) @ X1.T @ y, columns = ['Coef.'] )
        
        names.insert(0,"Intercept")
        beta_ols[''] = names
        beta_ols.set_index('', inplace=True)

    else:
        
        beta_ols = pd.DataFrame( np.linalg.inv(X.T @ X) @ X.T @ y, columns = ['Coef.'] )
        beta_ols[''] = names
        beta_ols.set_index('', inplace=True)  
    
    return beta_ols


In [8]:
beta_OLS( X  , y)

,Coef.
,
Intercept,-76.259257
lnq,-1.080425
(lnq)^2,0.026489
(lnq)(lnp1),0.131041
(lnq)(lnp2),0.058652
(lnq)(lnp3),0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797


In [9]:
beta_OLS( X  , y, intercept = False)

,Coef.
,
lnq,-1.070406
(lnq)^2,0.025880
(lnq)(lnp1),0.129652
(lnq)(lnp2),0.059482
(lnq)(lnp3),0.042202
lnp1,1.034946
lnp2,-1.331499
lnp3,-0.290481
(lnp1)^2,-0.301938


2. Generate a new function named **var_OLS**. This function must return the estimated variance for OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated variance does not include **intercept** in the `X` regressor. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The output of the function should look like `var_OLS_output` where the columns and the index use the name of `X` columns.  Apply your function to find $\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)})$ of the equation below. Use greene data.  **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{OLS}
$$


$$
\widehat{\sigma}^2_{OLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1)}
$$


$$
\begin{aligned}
\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)}) = 
\sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [10]:
file_to_open = open( r'../../_data/var_OLS_output.pkl' , "rb" )
var_OLS_output = pickle.load( file_to_open )
var_OLS_output

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,1465.361133,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932570,-8.392678,-128.192047,23.761713,-0.533889,7.081277,1.980507,21.804881,0.394870
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.030641,0.000762,-0.001796,-0.030093,0.035093,-0.007141
(lnq)^2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000086,0.000026,-0.000120,-0.000198,-0.000028,-0.000097
(lnq)(lnp1),-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.003923,0.000038,0.000571,0.003700,-0.002282,-0.000170
(lnq)(lnp2),0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.001175,-0.000340,-0.000205,-0.000340,-0.000289,-0.002751
(lnq)(lnp3),0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000421,-0.000066,-0.000134,0.000204,-0.003074,0.004424
lnp1,-262.932569,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194722,-1.253531,19.099543,-4.777971,0.140987,-0.840307,0.076832,-3.576317,0.264314
lnp2,-8.392678,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.323286,-0.117293,-0.066294,-1.370100,0.398963,-0.734776
lnp3,-128.192047,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099543,-0.214871,20.513714,-1.410427,0.047330,-1.526100,0.072458,-3.111107,-0.145182
(lnp1)^2,23.761713,0.030641,-0.000086,-0.003923,0.001175,0.000421,-4.777971,0.323286,-1.410427,0.479422,-0.012318,0.045414,-0.053651,0.281523,-0.026624


In [11]:
def var_OLS( X : pd.DataFrame , y : pd.Series , intercept = True ):
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( 'X is not a pd.DataFrame' )
    
    if not isinstance( y, pd.Series ):
        raise TypeError( 'y is not a pd.Series' )
    
    if not isinstance( intercept, bool ):
        raise TypeError( 'intercept is not a bool' )
        
        
    names = list(X.columns)


    if (intercept == True):

        X1 = np.hstack(( np.ones(len(X)).reshape(-1, 1 ) , X)) 
    
        beta_ols =  np.linalg.inv(X1.T @ X1) @ X1.T @ y
        
        # get predictions
        y_pred = np.dot( X1 , beta_ols)

        sr2 = np.sum( (y - y_pred) ** 2 ) / ( len( y ) - len(  beta_ols ) )

        var_beta = sr2 * np.linalg.inv(X1.T @ X1)
        
        #dataframe
        names.insert(0,"Intercept")
        
        var_beta = pd.DataFrame(var_beta, columns = names)
        
        var_beta[''] = names
        var_beta .set_index('', inplace=True)

        
    else:
        
        beta_ols = np.linalg.inv(X.T @ X) @ X.T @ y
        
         # get predictions
        y_pred = np.dot(X ,beta_ols)
        
        sr2 = np.sum( (y - y_pred) ** 2 ) / ( len( y ) - len(  beta_ols ) )

        var_beta = sr2 * np.linalg.inv(X.T @ X)
        
        #dataframe
        var_beta = pd.DataFrame(var_beta, columns = names)
        
        var_beta[''] = names
        var_beta .set_index('', inplace=True)
        
    return var_beta

In [12]:
var_OLS( X  , y)

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
,,,,,,,,,,,,,,,
Intercept,1465.361096,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932562,-8.392679,-128.192044,23.761712,-0.533889,7.081277,1.980507,21.804880,0.394870
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.030641,0.000762,-0.001796,-0.030093,0.035093,-0.007141
(lnq)^2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000086,0.000026,-0.000120,-0.000198,-0.000028,-0.000097
(lnq)(lnp1),-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.003923,0.000038,0.000571,0.003700,-0.002282,-0.000170
(lnq)(lnp2),0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.001175,-0.000340,-0.000205,-0.000340,-0.000289,-0.002751
(lnq)(lnp3),0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000421,-0.000066,-0.000134,0.000204,-0.003074,0.004424
lnp1,-262.932562,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-4.777971,0.140987,-0.840307,0.076832,-3.576317,0.264314
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.323286,-0.117293,-0.066294,-1.370100,0.398963,-0.734776
lnp3,-128.192044,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-1.410427,0.047330,-1.526100,0.072458,-3.111107,-0.145182


In [13]:
var_OLS( X  , y, False)

,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
,,,,,,,,,,,,,,
lnq,0.143916,0.000316,-0.013733,0.000162,-0.006220,-0.154803,0.152503,-0.075394,0.028086,0.000849,-0.002783,-0.030979,0.032892,-0.007341
(lnq)^2,0.000316,0.000006,-0.000037,-0.000005,-0.000011,-0.000620,0.001005,-0.000165,0.000106,0.000022,-0.000064,-0.000186,0.000149,-0.000095
(lnq)(lnp1),-0.013733,-0.000037,0.001495,-0.000171,0.000356,0.017120,-0.015595,0.003160,-0.003562,0.000029,0.000714,0.003813,-0.001924,-0.000166
(lnq)(lnp2),0.000162,-0.000005,-0.000171,0.000666,-0.000193,-0.005543,0.003468,0.009624,0.000935,-0.000341,-0.000288,-0.000369,-0.000537,-0.002812
(lnq)(lnp3),-0.006220,-0.000011,0.000356,-0.000193,0.000905,0.006897,-0.008805,0.003804,-0.000225,-0.000053,-0.000332,0.000154,-0.003738,0.004505
lnp1,-0.154803,-0.000620,0.017120,-0.005543,0.006897,3.078406,-2.816333,-3.982649,-0.524965,0.046122,0.439171,0.441108,0.343105,0.342076
lnp2,0.152503,0.001005,-0.015595,0.003468,-0.008805,-2.816333,8.196230,-0.968641,0.468849,-0.122832,-0.026268,-1.386768,0.534647,-0.747615
lnp3,-0.075394,-0.000165,0.003160,0.009624,0.003804,-3.982649,-0.968641,9.490986,0.682061,0.000637,-0.925310,0.250781,-1.228395,-0.112919
(lnp1)^2,0.028086,0.000106,-0.003562,0.000935,-0.000225,-0.524965,0.468849,0.682061,0.096052,-0.003736,-0.070845,-0.087534,-0.073542,-0.033708


3. Generate a new function named as `reg_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the output does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The ouput of this function should look like `reg_OLS_ouput` Apply your function to the equation below. Use greene data.**Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{RLS}
$$


$$
\widehat{\sigma}^2_{RLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1 - M)}
$$

In [14]:
file_to_open = open( r'../../_data/reg_OLS_output.pkl' , "rb" )
reg_OLS_output = pickle.load( file_to_open )
reg_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-76.259259,38.280036,-1.992142,4.825953e-02,-151.927107,-0.591411
lnq,-1.080425,0.375545,-2.876952,4.630885e-03,-1.822762,-0.338088
(lnq)^2,0.026489,0.002357,11.240060,2.112040e-21,0.021831,0.031148
(lnq)(lnp1),0.131041,0.038284,3.422854,8.084865e-04,0.055365,0.206716
(lnq)(lnp2),0.058652,0.025549,2.295650,2.315002e-02,0.008149,0.109155
(lnq)(lnp3),0.040144,0.029791,1.347518,1.799447e-01,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,3.954949e-02,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,7.533796e-01,-6.513111,4.723645
lnp3,6.380797,4.529207,1.408811,1.610617e-01,-2.572051,15.333645
(lnp1)^2,-1.538527,0.692403,-2.222011,2.785341e-02,-2.907195,-0.169860


In [15]:
import scipy.stats as stats

In [16]:
def reg_OLS( X : pd.DataFrame, y : pd.Series, intercept = True ):
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( 'X is not a pd.DataFrame' )
    
    if not isinstance( y, pd.Series ):
        raise TypeError( 'y is not a pd.Series' )
    
    if not isinstance( intercept, bool ):
        raise TypeError( 'intercept is not a bool' )
        
    if (intercept == True):
        
        beta = beta_OLS( X, y, intercept = True )
        var = var_OLS( X, y, intercept = True )
               
        #Dataframe
        
        reg_OLS = pd.DataFrame( beta )
        
        #Std.Err
        var_cov = var.to_numpy() 
        
        diagonal = [var_cov[i][i] for i in range( len(var_cov[0]) )] #extract diagonal
        
        std_error = np.sqrt( np.array(diagonal) )

        reg_OLS[ 'Std.Err.' ] = std_error 

        #t
        reg_OLS[ 't' ] = reg_OLS[ 'Coef.' ] / reg_OLS[ 'Std.Err.' ]
        
        #p-value
        p = stats.t.sf( abs( reg_OLS[ 't' ] ), df = len( y ) - len( beta ) ) * 2
        
        reg_OLS[ 'P>|t|' ] = p
        
        #[0.025        
        reg_OLS['[0.025'] = reg_OLS[ 'Coef.' ] - stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_OLS[ 'Std.Err.' ]
        
        #0.975]        
        reg_OLS['0.975]'] = reg_OLS[ 'Coef.' ] + stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_OLS[ 'Std.Err.' ]

        
    else:
        
        beta = beta_OLS( X, y, intercept = False )
        var = var_OLS( X, y, intercept = False )
            
        reg_OLS = pd.DataFrame( beta )
        
        #Std.Err
        var_cov = var.to_numpy() 
        
        diagonal = [var_cov[i][i] for i in range( len(var_cov[0]) )] #extract diagonal
        
        std_error = np.sqrt( np.array(diagonal) )

        reg_OLS[ 'Std.Err.' ] = std_error 
        
        #t
        reg_OLS[ 't' ] = reg_OLS[ 'Coef.' ] / reg_OLS[ 'Std.Err.' ]
        
        #p-value
        p = stats.t.sf( abs( reg_OLS[ 't' ] ), df = len( y ) - len( beta ) ) * 2
        
        reg_OLS[ 'P>|t|' ] = p
        
        #[0.025        
        reg_OLS['[0.025'] = reg_OLS[ 'Coef.' ] - stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_OLS[ 'Std.Err.' ]
        
        #0.975]        
        reg_OLS['0.975]'] = reg_OLS[ 'Coef.' ] + stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_OLS[ 'Std.Err.' ]

        
    return reg_OLS

In [17]:
reg_OLS(X, y)

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
,,,,,,
Intercept,-76.259257,38.280035,-1.992142,4.825953e-02,-151.927104,-0.591410
lnq,-1.080425,0.375545,-2.876952,4.630885e-03,-1.822762,-0.338088
(lnq)^2,0.026489,0.002357,11.240060,2.112040e-21,0.021831,0.031148
(lnq)(lnp1),0.131041,0.038284,3.422854,8.084865e-04,0.055365,0.206716
(lnq)(lnp2),0.058652,0.025549,2.295650,2.315002e-02,0.008149,0.109155
(lnq)(lnp3),0.040144,0.029791,1.347518,1.799447e-01,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,3.954950e-02,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,7.533796e-01,-6.513110,4.723645
lnp3,6.380797,4.529207,1.408811,1.610617e-01,-2.572051,15.333644


In [18]:
reg_OLS(X, y, False )

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
,,,,,,
lnq,-1.070406,0.379362,-2.821591,5.454148e-03,-1.820244,-0.320567
(lnq)^2,0.025880,0.002361,10.962820,1.033805e-20,0.021214,0.030547
(lnq)(lnp1),0.129652,0.038670,3.352769,1.022377e-03,0.053218,0.206087
(lnq)(lnp2),0.059482,0.025808,2.304809,2.260664e-02,0.008471,0.110493
(lnq)(lnp3),0.042202,0.030079,1.403049,1.627547e-01,-0.017251,0.101655
lnp1,1.034946,1.754539,0.589868,5.562035e-01,-2.433031,4.502924
lnp2,-1.331499,2.862906,-0.465086,6.425721e-01,-6.990247,4.327250
lnp3,-0.290481,3.080744,-0.094289,9.250106e-01,-6.379803,5.798842
(lnp1)^2,-0.301938,0.309922,-0.974238,3.315723e-01,-0.914521,0.310646


4. Generate a new function named as `restricted_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`,  `L`, `r`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series**,  `L` (coef) must be a **np.array**, `r` must be a **np.array**,  and `intercept` can be `True` or `False`.  When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The ouput of this function should look like `restricted_OLS_ouput`. Apply your function to the equation below. Use greene data.
 **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\hat{\boldsymbol{\beta}}^{(RLS)} =\hat{\boldsymbol{\beta}}^{(OLS)} - \left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \left( \boldsymbol{L}\left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \right)^{-1} \left( \boldsymbol{L}\widehat{\boldsymbol{\beta}}^{(OLS)} - \boldsymbol{r} \right)
$$

$$
\begin{aligned}
\mathbb{V}{\rm ar} \left(\widehat{\boldsymbol{\beta}}^{(RLS)} \right) 
&= \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1} - \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}  \mathbf{L}^\top \left( \mathbf{L}\left( \mathbf{X}^\top \mathbf{X} \right)^{-1} \mathbf{L}^\top \right)^{-1}  \mathbf{L} \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

ST: 

<br>
$$
\begin{aligned} 
\beta_{1} + \beta_{2} + \beta_{3} &= 1 \\
\beta_{q1} + \beta_{q2} + \beta_{q3} &= 0 \\
\beta_{11} + \beta_{12} + \beta_{13} &= 0 \\
\beta_{21} + \beta_{22} + \beta_{23} &= 0 \\
\beta_{31} + \beta_{32} + \beta_{33} &= 0 \\
\beta_{ij} = \beta_{ji}
\end{aligned}
$$

In [19]:
L = np.array( ( [ 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1 ] ) )

r = np.array(  [  1,  0,  0,  0,  0  ]  ).reshape( -1 , 1 )

In [20]:
file_to_open = open( r'../../_data/restricted_OLS_output.pkl' , "rb" )
restricted_OLS_output = pickle.load( file_to_open )
restricted_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-7.111200,2.755866,-2.580387,1.091364e-02,-12.560384,-1.662016
lnq,0.461368,0.119721,3.853679,1.776124e-04,0.224642,0.698094
(lnq)^2,0.029978,0.002121,14.134763,1.312279e-28,0.025784,0.034171
(lnq)(lnp1),-0.004354,0.022880,-0.190311,8.493452e-01,-0.049596,0.040887
(lnq)(lnp2),0.036778,0.024201,1.519668,1.308823e-01,-0.011075,0.084631
(lnq)(lnp3),-0.032424,0.021419,-1.513780,1.323685e-01,-0.074775,0.009928
lnp1,0.215566,1.015206,0.212338,8.321570e-01,-1.791804,2.222936
lnp2,0.402923,0.554162,0.727085,4.684055e-01,-0.692824,1.498670
lnp3,0.381511,0.531316,0.718048,4.739416e-01,-0.669063,1.432084
(lnp1)^2,-0.008712,0.140961,-0.061806,9.508067e-01,-0.287435,0.270011


In [21]:
def restricted_OLS( X : pd.DataFrame, y : pd.Series, L: np.ndarray , r: np.ndarray , intercept = True ):
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( 'X is not a pd.DataFrame' )
     
    if not isinstance( y, pd.Series ):
        raise TypeError( 'y is not a pd.Series' )
    
    if not isinstance( L, np.ndarray ):
        raise TypeError( 'L is not a np.array' )
        
    if not isinstance( r, np.ndarray ):
        raise TypeError( 'r is not a np.array' )
        
    if not isinstance( intercept, bool ):
        raise TypeError( 'intercept is not a bool' )
        
    #columns names X
    names = list(X.columns)
    
        
    if (intercept == True):
        
        beta = beta_OLS( X, y, intercept = True )
        
        beta = np.array(beta)
        
        X = np.hstack(( np.ones(len(X)).reshape(-1, 1 ) , X)) 
    
        #Then, we calculte the beta rls
        beta_rls = beta - np.linalg.inv(X.T @ X)  @ L.T @ np.linalg.inv(L @ np.linalg.inv(X.T @ X) @ L.T) @ (L@ beta - r)
        
        # Defining errors and variance
        res = np.array(y).reshape(-1, 1 ) - X @ beta
        sigma_sqr = ( res.T @ res) / (len(y)-(len(X[0])+1))

        # getting var_beta_rls
        var_beta_rls = sigma_sqr * np.linalg.inv(X.T @ X) - sigma_sqr * np.linalg.inv(X.T @ X) @ L.T @ np.linalg.inv( L @ np.linalg.inv(X.T @ X) @ L.T) @ L @ np.linalg.inv(X.T @ X)
        
        #Dataframe 
        reg_RLS = pd.DataFrame( beta_rls,  columns=['Coef.'] )
        
        #Std.Err        
        diagonal = [var_beta_rls[i][i] for i in range( len(var_beta_rls[0]) )] #extract diagonal
        
        std_error = np.sqrt( np.array(diagonal) )

        reg_RLS[ 'Std.Err.' ] = std_error 

        #t
        reg_RLS[ 't' ] = reg_RLS[ 'Coef.' ] / reg_RLS[ 'Std.Err.' ]
        
        #p-value
        p = stats.t.sf( abs( reg_RLS[ 't' ] ), df = len( y ) - len( beta ) ) * 2
        
        reg_RLS[ 'P>|t|' ] = p
        
        #[0.025        
        reg_RLS['[0.025'] = reg_RLS[ 'Coef.' ] - stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_RLS[ 'Std.Err.' ]
        
        #0.975]        
        reg_RLS['0.975]'] = reg_RLS[ 'Coef.' ] + stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_RLS[ 'Std.Err.' ]

        #index dataframe
        names.insert(0,"Intercept")
        reg_RLS[''] = names
        reg_RLS.set_index('', inplace=True)
        
    else:
        
        beta = beta_OLS( X, y, intercept = False )
            
        beta = np.array(beta)
        
        #new L
        L = np.delete(L, 0, 1)
        
        X = np.array(X)
            
        #Then, we calculte the beta rls
        beta_rls = beta - np.linalg.inv(X.T @ X)  @ L.T @ np.linalg.inv(L @ np.linalg.inv(X.T @ X) @ L.T) @ (L@ beta - r)
        
        # Defining errors and variance
        res = np.array(y).reshape(-1, 1 ) - X @ beta
        sigma_sqr = ( res.T @ res) / (len(y)-(len(X[0])+1))

        # getting var_beta_rls
        var_beta_rls = sigma_sqr * np.linalg.inv(X.T @ X) - sigma_sqr * np.linalg.inv(X.T @ X) @ L.T @ np.linalg.inv( L @ np.linalg.inv(X.T @ X) @ L.T) @ L @ np.linalg.inv(X.T @ X)
        
        #Dataframe 
        reg_RLS = pd.DataFrame( beta_rls,  columns=['Coef.'] )
        
        #Std.Err        
        diagonal = [var_beta_rls[i][i] for i in range( len(var_beta_rls[0]) )] #extract diagonal
        
        std_error = np.sqrt( np.array(diagonal) )

        reg_RLS[ 'Std.Err.' ] = std_error 

        #t
        reg_RLS[ 't' ] = reg_RLS[ 'Coef.' ] / reg_RLS[ 'Std.Err.' ]
        
        #p-value
        p = stats.t.sf( abs( reg_RLS[ 't' ] ), df = len( y ) - len( beta ) ) * 2
        
        reg_RLS[ 'P>|t|' ] = p
        
        #[0.025        
        reg_RLS['[0.025'] = reg_RLS[ 'Coef.' ] - stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_RLS[ 'Std.Err.' ]
        
        #0.975]        
        reg_RLS['0.975]'] = reg_RLS[ 'Coef.' ] + stats.t.ppf(q = 1 - 0.05 / 2, df = len(y)-len(beta)) * reg_RLS[ 'Std.Err.' ]

        #index Dataframe        
        reg_RLS[''] = names
        reg_RLS.set_index('', inplace=True)


        
        
    return reg_RLS

In [22]:
restricted_OLS( X , y , L , r , intercept = True )

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
,,,,,,
Intercept,-7.111200,2.765553,-2.571348,1.115123e-02,-12.577847,-1.644553
lnq,0.461368,0.120142,3.840181,1.841235e-04,0.223884,0.698852
(lnq)^2,0.029978,0.002128,14.085254,8.278278e-29,0.025771,0.034185
(lnq)(lnp1),-0.004354,0.022961,-0.189644,8.498568e-01,-0.049741,0.041032
(lnq)(lnp2),0.036778,0.024286,1.514345,1.321455e-01,-0.011229,0.084785
(lnq)(lnp3),-0.032424,0.021494,-1.508478,1.336384e-01,-0.074911,0.010064
lnp1,0.215566,1.018774,0.211594,8.327252e-01,-1.798237,2.229369
lnp2,0.402923,0.556110,0.724538,4.699191e-01,-0.696335,1.502181
lnp3,0.381511,0.533184,0.715533,4.754465e-01,-0.672430,1.435451


In [23]:
restricted_OLS( X , y , L , r , intercept =False )

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
,,,,,,
lnq,0.380786,0.117170,3.249868,1.437421e-03,0.149191,0.612380
(lnq)^2,0.030064,0.002150,13.984445,1.304166e-28,0.025815,0.034313
(lnq)(lnp1),0.008783,0.022614,0.388364,6.983205e-01,-0.035916,0.053481
(lnq)(lnp2),0.018255,0.023431,0.779102,4.371980e-01,-0.028058,0.064568
(lnq)(lnp3),-0.027038,0.021611,-1.251113,2.129222e-01,-0.069753,0.015678
lnp1,-2.340761,0.224875,-10.409156,2.880829e-19,-2.785244,-1.896278
lnp2,1.755077,0.182782,9.602030,3.530482e-17,1.393795,2.116359
lnp3,1.585684,0.257512,6.157715,6.966544e-09,1.076693,2.094676
(lnp1)^2,0.323571,0.058128,5.566553,1.232571e-07,0.208677,0.438465


5. Generate a new class named `RegClass`. The initial attributes of your class must be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. Also, you must specify the type of your class' parameters, and output and It must raise an error if the inputs do not meet the requirements.  This class should have two methods: `reg_OLS` and `restricted_OLS` (You can add more if you think it is necessary). These methods should return the same output as the previously defined functions with the same name. The method `reg_OLS` does not need an extra parameter to be executed. After executing the method `reg_OLS`, any user should access the following attributes: `beta_OLS` and `var_OLS`. They should look like `beta_OLS_output` and `var_OLS_output`. The `restricted_OLS` method needs two extra parameters before being executed, `L` and `r`. They refer to the restrictions. After running the method `restricted_OLS`, any user should access the following attributes: `beta_OLS`, `var_OLS`, `rest_beta_OLS`, and `rest_var_OLS`. These attributes refer to the beta and variance estimated using OLS and restricted OLS. They should look like  `beta_OLS_output` and `var_OLS_output`. Fix all the methods and attributes. No user should be able to change them. **Hint: Use [lecture 6](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f362f4c6563747572655f362e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_6%2FLecture_6.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96#6.3.).**

First, we define 2 functions that will help us when we define RegClass

In [24]:
def beta_RLS( X : pd.DataFrame, y : pd.Series, L: np.ndarray , r: np.ndarray , intercept = True ):
    
    #columns names X
    names = list(X.columns)
    
    if (intercept == True):
                
        beta = beta_OLS( X, y, intercept = True )
        
        beta = np.array(beta)
        
        X = np.hstack(( np.ones(len(X)).reshape(-1, 1 ) , X)) 
    
        #Then, we calculte the beta rls
        beta_rls = beta - np.linalg.inv(X.T @ X)  @ L.T @ np.linalg.inv(L @ np.linalg.inv(X.T @ X) @ L.T) @ (L@ beta - r)
        
        beta_rls = pd.DataFrame(beta_rls, columns = ['Coef.'])
        
        #index dataframe
        names.insert(0,"Intercept")
        beta_rls[''] = names
        beta_rls.set_index('', inplace=True)
        
    else:
        
        beta = beta_OLS( X, y, intercept = False )
            
        beta = np.array(beta)
        
        #new L
        L = np.delete(L, 0, 1)
        
        X = np.array(X)
            
        #Then, we calculte the beta rls
        beta_rls = beta - np.linalg.inv(X.T @ X)  @ L.T @ np.linalg.inv(L @ np.linalg.inv(X.T @ X) @ L.T) @ (L@ beta - r)
        
        beta_rls = pd.DataFrame(beta_rls, columns = ['Coef.'])
        
        #index Dataframe        
        beta_rls[''] = names
        beta_rls.set_index('', inplace=True)
        
    return beta_rls

In [25]:
def var_RLS( X : pd.DataFrame, y : pd.Series, L: np.ndarray , r: np.ndarray , intercept = True ):
    
    names = list(X.columns)
        
    if (intercept == True):
        
        beta = beta_OLS( X, y, intercept = True )
        
        beta = np.array(beta)
        
        X = np.hstack(( np.ones(len(X)).reshape(-1, 1 ) , X)) 
        
        # Defining errors and variance
        res = np.array(y).reshape(-1, 1 ) - X @ beta
        sigma_sqr = ( res.T @ res) / (len(y)-(len(X[0])+1))

        # getting var_beta_rls
        var_beta_rls = sigma_sqr * np.linalg.inv(X.T @ X) - sigma_sqr * np.linalg.inv(X.T @ X) @ L.T @ np.linalg.inv( L @ np.linalg.inv(X.T @ X) @ L.T) @ L @ np.linalg.inv(X.T @ X)
        
        #dataframe
        names.insert(0,"Intercept")
        
        var_beta_rls = pd.DataFrame(var_beta_rls, columns = names)
        
        var_beta_rls[''] = names
        var_beta_rls.set_index('', inplace=True)
        
    else:
        
        beta = beta_OLS( X, y, intercept = False )
            
        beta = np.array(beta)
        
        #new L
        L = np.delete(L, 0, 1)
        
        X = np.array(X)
            
        # Defining errors and variance
        res = np.array(y).reshape(-1, 1 ) - X @ beta
        sigma_sqr = ( res.T @ res) / (len(y)-(len(X[0])+1))

        # getting var_beta_rls
        var_beta_rls = sigma_sqr * np.linalg.inv(X.T @ X) - sigma_sqr * np.linalg.inv(X.T @ X) @ L.T @ np.linalg.inv( L @ np.linalg.inv(X.T @ X) @ L.T) @ L @ np.linalg.inv(X.T @ X)
        
        #dataframe
        var_beta_rls = pd.DataFrame(var_beta_rls, columns = names)
        
        var_beta_rls[''] = names
        var_beta_rls.set_index('', inplace=True)
        
    return var_beta_rls

Now, we define RegClass

In [26]:
class RegClass:
    
    def __init__ (self, X : pd.DataFrame, y : pd.Series, L: np.ndarray , r: np.ndarray , intercept = True):
        
        __slots__ = [ 'X', 'y', 'intercept', 'beta_OLS', 'var_OLS', 'rest_beta_OLS', 'rest_var_OLS', ]
        
        if not isinstance( X, pd.DataFrame ):
            raise TypeError( 'X is not a pd.DataFrame' )
            
        if not isinstance( y, pd.Series ):
            raise TypeError( 'y is not a pd.Series' )
            
        if not isinstance( L, np.ndarray ):
            raise TypeError( 'L is not a np.array' )
        
        if not isinstance( r, np.ndarray ):
            raise TypeError( 'r is not a np.array' )
    
        if not isinstance( intercept, bool ):
            raise TypeError( 'intercept is not a bool' )
            
        self.X = X
        self.y = y
        self.L = L
        self.r = r
        self.intercept = intercept
    
    def reg_OLS(self):
        
        self.beta_OLS = beta_OLS( self.X, self.y, intercept = self.intercept )
        self.var_OLS = var_OLS( self.X, self.y, intercept = self.intercept )
        
        return self.beta_OLS, self.var_OLS
    
    def restricted_OLS(self):
        
        self.beta_OLS = beta_OLS( self.X, self.y, intercept = self.intercept )
        self.var_OLS = var_OLS( self.X, self.y, intercept = self.intercept )

        self.rest_beta_OLS = beta_RLS( self.X, self.y, self.L, self.r, intercept = self.intercept )
        self.rest_var_OLS = var_RLS( self.X, self.y, self.L, self.r, intercept = self.intercept )
        
        return self.beta_OLS, self.var_OLS, self.rest_beta_OLS, self.rest_var_OLS      

In [27]:
reg_class = RegClass(X, y, L, r, False)

In [28]:
reg_class.reg_OLS()

(                 Coef.
                       
 lnq          -1.070406
 (lnq)^2       0.025880
 (lnq)(lnp1)   0.129652
 (lnq)(lnp2)   0.059482
 (lnq)(lnp3)   0.042202
 lnp1          1.034946
 lnp2         -1.331499
 lnp3         -0.290481
 (lnp1)^2     -0.301938
 (lnp2)^2     -0.101290
 (lnp3)^2      0.041104
 (lnp1)(lnp2)  0.427634
 (lnp1)(lnp3) -0.009347
 (lnp2)(lnp3) -0.027500,
                    lnq   (lnq)^2  (lnq)(lnp1)  (lnq)(lnp2)  (lnq)(lnp3)  \
                                                                           
 lnq           0.143916  0.000316    -0.013733     0.000162    -0.006220   
 (lnq)^2       0.000316  0.000006    -0.000037    -0.000005    -0.000011   
 (lnq)(lnp1)  -0.013733 -0.000037     0.001495    -0.000171     0.000356   
 (lnq)(lnp2)   0.000162 -0.000005    -0.000171     0.000666    -0.000193   
 (lnq)(lnp3)  -0.006220 -0.000011     0.000356    -0.000193     0.000905   
 lnp1         -0.154803 -0.000620     0.017120    -0.005543     0.006897   
 lnp2  

In [29]:
reg_class.reg_OLS()[0] 

,Coef.
,
lnq,-1.070406
(lnq)^2,0.025880
(lnq)(lnp1),0.129652
(lnq)(lnp2),0.059482
(lnq)(lnp3),0.042202
lnp1,1.034946
lnp2,-1.331499
lnp3,-0.290481
(lnp1)^2,-0.301938


In [30]:
reg_class.reg_OLS()[1]

,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
,,,,,,,,,,,,,,
lnq,0.143916,0.000316,-0.013733,0.000162,-0.006220,-0.154803,0.152503,-0.075394,0.028086,0.000849,-0.002783,-0.030979,0.032892,-0.007341
(lnq)^2,0.000316,0.000006,-0.000037,-0.000005,-0.000011,-0.000620,0.001005,-0.000165,0.000106,0.000022,-0.000064,-0.000186,0.000149,-0.000095
(lnq)(lnp1),-0.013733,-0.000037,0.001495,-0.000171,0.000356,0.017120,-0.015595,0.003160,-0.003562,0.000029,0.000714,0.003813,-0.001924,-0.000166
(lnq)(lnp2),0.000162,-0.000005,-0.000171,0.000666,-0.000193,-0.005543,0.003468,0.009624,0.000935,-0.000341,-0.000288,-0.000369,-0.000537,-0.002812
(lnq)(lnp3),-0.006220,-0.000011,0.000356,-0.000193,0.000905,0.006897,-0.008805,0.003804,-0.000225,-0.000053,-0.000332,0.000154,-0.003738,0.004505
lnp1,-0.154803,-0.000620,0.017120,-0.005543,0.006897,3.078406,-2.816333,-3.982649,-0.524965,0.046122,0.439171,0.441108,0.343105,0.342076
lnp2,0.152503,0.001005,-0.015595,0.003468,-0.008805,-2.816333,8.196230,-0.968641,0.468849,-0.122832,-0.026268,-1.386768,0.534647,-0.747615
lnp3,-0.075394,-0.000165,0.003160,0.009624,0.003804,-3.982649,-0.968641,9.490986,0.682061,0.000637,-0.925310,0.250781,-1.228395,-0.112919
(lnp1)^2,0.028086,0.000106,-0.003562,0.000935,-0.000225,-0.524965,0.468849,0.682061,0.096052,-0.003736,-0.070845,-0.087534,-0.073542,-0.033708


In [31]:
reg_class.restricted_OLS()

(                 Coef.
                       
 lnq          -1.070406
 (lnq)^2       0.025880
 (lnq)(lnp1)   0.129652
 (lnq)(lnp2)   0.059482
 (lnq)(lnp3)   0.042202
 lnp1          1.034946
 lnp2         -1.331499
 lnp3         -0.290481
 (lnp1)^2     -0.301938
 (lnp2)^2     -0.101290
 (lnp3)^2      0.041104
 (lnp1)(lnp2)  0.427634
 (lnp1)(lnp3) -0.009347
 (lnp2)(lnp3) -0.027500,
                    lnq   (lnq)^2  (lnq)(lnp1)  (lnq)(lnp2)  (lnq)(lnp3)  \
                                                                           
 lnq           0.143916  0.000316    -0.013733     0.000162    -0.006220   
 (lnq)^2       0.000316  0.000006    -0.000037    -0.000005    -0.000011   
 (lnq)(lnp1)  -0.013733 -0.000037     0.001495    -0.000171     0.000356   
 (lnq)(lnp2)   0.000162 -0.000005    -0.000171     0.000666    -0.000193   
 (lnq)(lnp3)  -0.006220 -0.000011     0.000356    -0.000193     0.000905   
 lnp1         -0.154803 -0.000620     0.017120    -0.005543     0.006897   
 lnp2  